In [7]:
import os
import sys
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import desc
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

ratings = spark.read.csv("data/ratings.csv", header=True, inferSchema=True)
(training, test) = ratings.randomSplit([0.8, 0.2], 123)
als = ALS(maxIter=5, rank=70, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
            coldStartStrategy="drop")
als.setSeed(123)
model = als.fit(training)
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


# Calculate the sparsity of the user-item rating matrix
# num_ratings = df.select("rating").count()
# num_users = df.select("userId").distinct().count()
# num_items = df.select("movieId").distinct().count()
# num_elements = num_users * num_items
# num_non_zero = df.select("rating").na.drop().count()
# sparsity = (num_non_zero / num_elements)*100




# # Set the minimum number of ratings per user and per item
# min_ratings_user = 20
# min_ratings_item = 10

# # Filter out users with fewer than min_ratings_user ratings
# user_counts = df.groupBy("userId").count().filter(col("count") >= min_ratings_user)
# df = df.join(user_counts, "userId", "inner")

# # Filter out items with fewer than min_ratings_item ratings
# item_counts = df.groupBy("movieId").count().filter(col("count") >= min_ratings_item)
# df = df.join(item_counts, "movieId", "inner")

23/03/20 17:09:04 WARN BlockManager: Block rdd_532_2 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_532_3 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_532_0 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_532_7 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_533_0 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_533_2 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_533_7 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 WARN BlockManager: Block rdd_533_3 could not be removed as it was not found on disk or in memory
23/03/20 17:09:04 ERROR Executor: Exception in task 0.0 in stage 267.0 (TID 630)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/q0/3kkrdgjd2vb_xbkj2f0m3ss40000gn/T/ipykernel_23662/2343868668.py", line 23, in <module>
    model = als.fit(training)
            ^^^^^^^^^^^^^^^^^
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/pyspark/ml/base.py", line 205, in fit
    return self._fit(dataset)
           ^^^^^^^^^^^^^^^^^^
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/pyspark/ml/wrapper.py", line 383, in _fit
    java_model = self._fit_java(dataset)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-pa

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/martinsenecal/Code/BigData-471/project/Movie-recommender/venv/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
